In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from docx import Document

from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier, XGBRegressor
import pandas as pd
import numpy as np

from linearmodels.panel import PanelOLS
import statsmodels.formula.api as smf
import statsmodels.api as sm


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mtick
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Base final da amostra com categoricas criadas

In [2]:
# Import final data base de inscritos presentes em todas as provas sem missing ~32 MM
df = pd.read_csv('df_basepronta2.csv', sep = ';')

print(df.shape)
#(1529177, 57)

(1529177, 85)


In [3]:
list(df)

['NU_ANO',
 'NU_IDADE',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'NU_NOTA_REDACAO',
 'Q005',
 'concluinte_3ano',
 'baixarenda',
 'marcacao',
 'cat_Q001',
 'cat_Q002',
 'cat_Q003',
 'cat_Q004',
 'cat_Q006',
 'cat_Q007',
 'cat_Q008',
 'cat_Q009',
 'cat_Q010',
 'cat_Q011',
 'cat_Q012',
 'cat_Q013',
 'cat_Q014',
 'cat_Q015',
 'cat_Q016',
 'cat_Q017',
 'cat_Q019',
 'cat_Q022',
 'cat_Q024',
 'cat_Q018',
 'cat_Q020',
 'cat_Q021',
 'cat_Q023',
 'cat_Q025',
 'FEMININO',
 'REG_UF',
 'cat_Norte',
 'cat_Nordeste',
 'cat_Sul',
 'cat_CentroOeste',
 'cat_Sudeste',
 'tem_TIC',
 'tem_TICv2',
 'tem_TICv3',
 'TIC',
 'I',
 'C',
 'sóI',
 'sóC',
 'IeC',
 'NU_MEDIA',
 'RACA',
 'Branca',
 'Parda',
 'Raca_outra',
 'renda_0e1',
 'renda_15',
 'CentroOeste',
 'Nordeste',
 'Norte',
 'Sudeste',
 'Sul',
 'escpartic_outra',
 'escpubl',
 'Mae_ESup_mais',
 'Mae_Ens_fundamental2',
 'Mae_Ens_medio3',
 'Mae_desc_n

In [6]:
import pandas as pd

# Supondo que seu DataFrame se chame df e que a coluna 'NU_ANO' representa os anos

# Lista das variáveis de interesse
variaveis_de_interesse = [
    'I', 'C', 'sóI', 'sóC', 'IeC', 'NU_MEDIA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
    'NU_NOTA_REDACAO', 'FEMININO','Branca', 'Parda', 'Raca_outra', 'renda_0e1', 'renda_15', 'CentroOeste', 'Nordeste',
    'Norte',    'Sudeste', 'Sul', 'escpartic_outra', 'escpubl', 'Mae_ESup_mais', 'Mae_Ens_fundamental2', 
    'Mae_Ens_medio3', 'Mae_desc_nada0', 'Mae_fund_incompl1', 'Pai_ESup_mais', 'Pai_Ens_fundamental2', 
    'Pai_Ens_medio3', 'Pai_desc_nada0', 'Pai_fund_incompl1', 'tamfam3peq', 'tamfam5med', 'tamfam6gde',
    'infra_basicacompleta','infra_basicaincompleta'
]

# Agrupando por ano e calculando as estatísticas descritivas para as variáveis selecionadas
descricao_por_ano = df.groupby('NU_ANO')[variaveis_de_interesse].agg(['count', 'mean', 'std', 'min', 'median', 'max'])

# Renomear colunas para um formato mais legível (opcional)
descricao_por_ano.columns = ['_'.join(col).strip() for col in descricao_por_ano.columns.values]

# Visualizando a descrição por ano
print(descricao_por_ano)

# Se quiser salvar em um arquivo Excel ou CSV
descricao_por_ano.to_excel('descricao_por_ano.xlsx')



        NU_IDADE_count  NU_IDADE_mean  NU_IDADE_std  NU_IDADE_min  \
NU_ANO                                                              
2015            195417      18.685872      4.131031     12.000000   
2016            219986      18.647796      3.971972     12.000000   
2017            209467      18.562084      3.779666     10.000000   
2018            196173      18.405953      3.394957     12.000000   
2019            189130      18.320785      3.234319     12.000000   
2020            115977      18.129319      2.752025     16.000000   
2021            112647      18.079256      2.383534     16.000000   
2022            135328      18.072542      2.485045     16.000000   
2023            155052      18.117025      2.615155     16.000000   

        NU_IDADE_median  NU_IDADE_max  TP_ESTADO_CIVIL_count  \
NU_ANO                                                         
2015          18.000000     83.000000                 195417   
2016          18.000000     73.000000           

In [4]:
# Lista das variáveis de interesse
variaveis_de_interesse = [
    'I', 'C', 'sóI', 'sóC', 'IeC', 'NU_MEDIA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
    'NU_NOTA_REDACAO', 'FEMININO','Branca', 'Parda', 'Raca_outra', 'renda_0e1', 'renda_15', 'CentroOeste', 'Nordeste',
    'Norte',    'Sudeste', 'Sul', 'escpartic_outra', 'escpubl', 'Mae_ESup_mais', 'Mae_Ens_fundamental2', 
    'Mae_Ens_medio3', 'Mae_desc_nada0', 'Mae_fund_incompl1', 'Pai_ESup_mais', 'Pai_Ens_fundamental2', 
    'Pai_Ens_medio3', 'Pai_desc_nada0', 'Pai_fund_incompl1', 'tamfam3peq', 'tamfam5med', 'tamfam6gde',
    'infra_basicacompleta','infra_basicaincompleta'
]

# Agrupando por ano e calculando as estatísticas descritivas para as variáveis selecionadas
descricao = df[variaveis_de_interesse].agg(['count', 'mean', 'std', 'min', 'median', 'max'])

# Renomear colunas para um formato mais legível (opcional)
descricao.columns = ['_'.join(col).strip() for col in descricao.columns.values]

# Visualizando a descrição
print(descricao)
# Se quiser salvar em um arquivo Excel ou CSV
descricao.to_excel('descricao_por_ano_amostratotal.xlsx')


                    I              C          s_ó_I          s_ó_C  \
count  1529177.000000 1529177.000000 1529177.000000 1529177.000000   
mean         0.638239       0.368264       0.314110       0.044136   
std          0.480510       0.482334       0.464161       0.205396   
min          0.000000       0.000000       0.000000       0.000000   
median       1.000000       0.000000       0.000000       0.000000   
max          1.000000       1.000000       1.000000       1.000000   

                I_e_C  N_U___M_E_D_I_A  N_U___N_O_T_A___C_N  \
count  1529177.000000   1529177.000000       1529177.000000   
mean         0.324129       481.328782           460.787984   
std          0.468048        54.599088            59.856072   
min          0.000000       316.625000           299.900000   
median       0.000000       475.000000           453.900000   
max          1.000000       687.325000           643.000000   

        N_U___N_O_T_A___C_H  N_U___N_O_T_A___L_C  N_U___N_O_T_A___M

## Incluir uma analise das medias das notas por disciplina que possuem ou não acesso a internet e ao computador e fazer referencia a literatura que essas descritivas colaboram!

In [5]:

# Criando a tabela cruzada com média de 'NU_MEDIA' para cada combinação de 'NU_ANO' e 'IeC'
crosstab_result = pd.crosstab(df['NU_ANO'], df['IeC'], values=df['NU_MEDIA'], aggfunc='mean')

# Renomeando as colunas para clareza
crosstab_result.columns = ['Media_NU_MEDIA_IeC_0', 'Media_NU_MEDIA_IeC_1']

# Resetando o índice para transformar o resultado em um DataFrame padrão
crosstab_result = crosstab_result.reset_index()

# Exibindo o resultado
print(crosstab_result)


   NU_ANO  Media_NU_MEDIA_IeC_0  Media_NU_MEDIA_IeC_1
0    2015            465.429199            484.814029
1    2016            471.318967            492.296273
2    2017            475.697377            499.107724
3    2018            492.455541            516.111629
4    2019            468.635106            494.803031
5    2020            467.402237            499.138876
6    2021            461.524581            492.109881
7    2022            476.913023            504.414322
8    2023            474.634032            504.217047


In [7]:
df.shape

(1529177, 85)

In [6]:
import pandas as pd

# Lista das colunas de notas
notas_colunas = ['NU_MEDIA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

# Inicializa um dicionário para armazenar os DataFrames
resultados = {}

# Cria a tabela cruzada para cada coluna de nota
for coluna in notas_colunas:
    crosstab_result = pd.crosstab(df['NU_ANO'], df['IeC'], values=df[coluna], aggfunc='mean')
    crosstab_result.columns = [f'Media_{coluna}_IeC_0', f'Media_{coluna}_IeC_1']
    crosstab_result = crosstab_result.reset_index()
    resultados[coluna] = crosstab_result

# Mescla todos os DataFrames em um único DataFrame
resultado_final = pd.merge(resultados['NU_MEDIA'], resultados['NU_NOTA_CN'], on='NU_ANO')
for coluna in notas_colunas[2:]:
    resultado_final = pd.merge(resultado_final, resultados[coluna], on='NU_ANO')

# Salvando o resultado em um arquivo Excel
resultado_final.to_excel('resultados_medianas_por_ano.xlsx', index=False)

# Exibindo o resultado
print(resultado_final)


   NU_ANO  Media_NU_MEDIA_IeC_0  Media_NU_MEDIA_IeC_1  Media_NU_NOTA_CN_IeC_0  \
0    2015            465.429199            484.814029              443.927451   
1    2016            471.318967            492.296273              449.574642   
2    2017            475.697377            499.107724              476.404725   
3    2018            492.455541            516.111629              459.855311   
4    2019            468.635106            494.803031              440.478413   
5    2020            467.402237            499.138876              450.392192   
6    2021            461.524581            492.109881              445.750568   
7    2022            476.913023            504.414322              461.841366   
8    2023            474.634032            504.217047              458.098432   

   Media_NU_NOTA_CN_IeC_1  Media_NU_NOTA_CH_IeC_0  Media_NU_NOTA_CH_IeC_1  \
0              460.505896              522.905576              541.009553   
1              465.420463          

In [4]:
descricao_por_ano = df.groupby(['NU_ANO', 'FEMININO']).agg({
    'NU_MEDIA': ['mean', 'size', 'min', 'median', 'max'],
    'NU_NOTA_CN': ['mean', 'size', 'min', 'median', 'max'],
    'NU_NOTA_CH': ['mean', 'size', 'min', 'median', 'max'],
    'NU_NOTA_LC': ['mean', 'size', 'min', 'median', 'max'],
    'NU_NOTA_MT': ['mean', 'size', 'min', 'median', 'max'],
    'NU_NOTA_REDACAO': ['mean', 'size', 'min', 'median', 'max'],
    'I': ['mean', 'size', 'min', 'median', 'max'],
    'C': ['mean', 'size', 'min', 'median', 'max'],
    'sóI': ['mean', 'size', 'min', 'median', 'max'],
    'sóC': ['mean', 'size', 'min', 'median', 'max'],
    'IeC': ['mean', 'size', 'min', 'median', 'max']
})

In [5]:
print(descricao_por_ano.head())

                  NU_MEDIA                                             \
                      mean       std        min     median        max   
NU_ANO FEMININO                                                         
2015   0        478.222720 53.814283 326.125000 473.225000 677.275000   
       1        470.170241 47.421667 327.000000 466.225000 682.625000   
2016   0        485.777765 51.985743 340.025000 478.700000 687.325000   
       1        475.069750 46.589353 339.200000 469.100000 683.250000   
2017   0        489.949645 54.303939 339.900000 484.050000 679.525000   

                NU_NOTA_CN                                             \
                      mean       std        min     median        max   
NU_ANO FEMININO                                                         
2015   0        461.482537 60.048996 335.700000 457.200000 643.000000   
       1        444.176191 53.902365 336.500000 438.700000 642.800000   
2016   0        463.689724 58.754208 343.500000 45

In [6]:
descricao_por_ano = df.groupby(['NU_ANO', 'FEMININO']).agg({
    'NU_MEDIA': ['mean', 'size', 'min', 'max'],
    'NU_NOTA_CN': ['mean', 'size', 'min', 'max'],
    'NU_NOTA_CH': ['mean', 'size', 'min', 'max'],
    'NU_NOTA_LC': ['mean', 'size', 'min', 'max'],
    'NU_NOTA_MT': ['mean', 'size', 'min', 'max'],
    'NU_NOTA_REDACAO': ['mean', 'size', 'min','max'],
    'I': ['mean', 'size', 'min', 'max'],
    'C': ['mean', 'size', 'min', 'max'],
    'sóI': ['mean', 'size', 'min', 'max'],
    'sóC': ['mean', 'size', 'min', 'max'],
    'IeC': ['mean', 'size', 'min', 'max']
})

In [7]:
print(descricao_por_ano.head())

                  NU_MEDIA                               NU_NOTA_CN          \
                      mean    size        min        max       mean    size   
NU_ANO FEMININO                                                               
2015   0        478.222720   71379 326.125000 677.275000 461.482537   71379   
       1        470.170241  124038 327.000000 682.625000 444.176191  124038   
2016   0        485.777765   82248 340.025000 687.325000 463.689724   82248   
       1        475.069750  137738 339.200000 683.250000 450.501135  137738   
2017   0        489.949645   79443 339.900000 679.525000 489.812414   79443   

                                      NU_NOTA_CH                     \
                       min        max       mean    size        min   
NU_ANO FEMININO                                                       
2015   0        335.700000 643.000000 535.951270   71379 341.200000   
       1        336.500000 642.800000 526.701783  124038 341.200000   
2016   0    

In [8]:
descricao_por_ano.to_excel('descricao_por_ano.xlsx', index=True, float_format="%.2f")